In [1]:
# create Vgg16 model

import numpy as np
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D

vgg_mean = np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape((1,1,3)) # BGR

def vgg_preprocess(x):
    x = x - vgg_mean 
    return x

model = Sequential()
model.add(Lambda(vgg_preprocess, input_shape=(224,224,3), output_shape=(224,224,3)))

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))


# Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# FC blocks
model.add(Flatten(name='flatten'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1000, activation='softmax'))

# load weights
fname = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'
FILE_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/'
model.load_weights(get_file(fname, FILE_PATH+fname, cache_subdir='models'))

model.summary()




Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
__________

In [2]:
# remove last layer from model
model.layers.pop()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
__________

In [3]:
# set all layers to trainable false
for layer in model.layers: layer.trainable=False
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
__________

In [4]:
# add only 2 classes
model.add(Dense(2, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
__________

In [6]:
# Now, start working on the data
# data location (a small sample of it)
path = '/home/ubuntu/fastai/courses/deeplearning1/nbs/data/dogscats/sample/'
model_path = path + 'models/'

batch_size = 32

from fcn_utils import get_batches, get_data

val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)

Found 40 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [7]:
target_size=(224,224)

batches = get_batches(path+'valid', shuffle=False, batch_size=1, class_mode=None, target_size=target_size)
val_data = np.concatenate([batches.next() for i in range(batches.samples)])

batches = get_batches(path+'train', shuffle=False, batch_size=1, class_mode=None, target_size=target_size)
trn_data = np.concatenate([batches.next() for i in range(batches.samples)])

Found 40 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [8]:
from sklearn.preprocessing import OneHotEncoder

def onehot(x):
      return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
    
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

from keras.preprocessing import image

gen = image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_labels, batch_size=batch_size, shuffle=True)
val_batches = gen.flow(val_data, val_labels, batch_size=batch_size, shuffle=False) # this does not touch the validation data

def fit_model(model, batches, val_batches, nb_epoch=1):
      model.fit_generator(batches, samples_per_epoch=batches.n, nb_epoch=nb_epoch, validation_data=val_batches, nb_val_samples=batches.n)


In [10]:
from keras.optimizers import SGD, RMSprop
opt = RMSprop(lr=0.1)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
fit_model(model, batches, val_batches, nb_epoch=10)        

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=6, epochs=10, validation_steps=200)`


Epoch 1/10
6/6 [==============================] - 146s - loss: 0.4082 - acc: 0.8480 - val_loss: 1.0679 - val_acc: 0.5750
Epoch 2/10
6/6 [==============================] - 146s - loss: 0.6402 - acc: 0.6835 - val_loss: 0.3801 - val_acc: 0.8750
Epoch 3/10
6/6 [==============================] - 146s - loss: 0.3374 - acc: 0.8882 - val_loss: 0.2420 - val_acc: 0.9500
Epoch 4/10
6/6 [==============================] - 146s - loss: 0.4172 - acc: 0.8512 - val_loss: 0.4286 - val_acc: 0.8000
Epoch 5/10
6/6 [==============================] - 146s - loss: 0.2787 - acc: 0.9150 - val_loss: 0.5442 - val_acc: 0.7750
Epoch 6/10
5/6 [========================>.....] - ETA: 1s - loss: 0.3751 - acc: 0.8562

KeyboardInterrupt: 